In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
DATA_DIR = "input_data"
today_str = datetime.now().strftime("%Y-%m-%d")
file_name = f"/kaggle/input/my-input-dataset/{today_str}_input_data.csv"
Clean_data = pd.read_csv(file_name)

In [ ]:
Clean_data.head(10)

In [ ]:
print(Clean_data.columns)

In [ ]:
Clean_data = Clean_data.drop('Time_x', axis=1)

In [ ]:

# Create 'group' column (every 4 rows = 1 hour)
Clean_data['group'] = Clean_data.index // 4

# Group by 'group' and take averages for all MW columns
hourly_Data = Clean_data.groupby('group').agg({
    'Date': 'first',
    'Hour': 'first',
    'Purchase Bid (MW)': 'mean',
    'Total Sell Bid (MW)': 'mean',
    'Sell bid Solar (MW)': 'mean',
    'Sell bid Non-Solar (MW)': 'mean',
    'Sell bid Hydro (MW)': 'mean',
    'MCV Total (MW)': 'mean',
    'MCV Solar (MW)': 'mean',
    'MCV Non-Solar (MW)': 'mean',
    'MCV Hydro (MW)': 'mean',
    'MCP (Rs/MWh) ': 'mean',  # keep mean for MCP as well
}).reset_index(drop=True)

# --- Adjust Hour column (1–24, reset daily) ---
hourly_Data['Hour'] = (hourly_Data.groupby('Date').cumcount() % 24) + 1


In [ ]:
hourly_Data.shape

In [ ]:
print(hourly_Data.columns)

In [ ]:
hourly_Data.to_csv('df_Hourly.csv', index=False)

In [ ]:
# Convert 'Date' column to datetime objects
hourly_Data['Date'] = pd.to_datetime(hourly_Data['Date'])

# Combine 'Date' and 'Hour' to create a new 'Datetime' column
# We need to adjust the Hour since it's 1-24 and datetime expects 0-23 for hour
hourly_Data['Datetime'] = hourly_Data['Date'] + pd.to_timedelta(hourly_Data['Hour'] - 1, unit='h')

# Display the first few rows with the new 'Datetime' column
print(hourly_Data[['Date', 'Hour', 'Datetime']].head())

In [ ]:
# Drop the original 'Date' and 'Hour' columns
hourly_Data = hourly_Data.drop(['Date'], axis=1)

# Set the 'Datetime' column as the index
hourly_Data = hourly_Data.set_index('Datetime')

In [ ]:
hourly_Data.head()

##feature engineering

In [ ]:
daily_avg = hourly_Data.copy()

In [ ]:
daily_avg['Day_of_Week'] = daily_avg.index.dayofweek
daily_avg['Month'] = daily_avg.index.month

In [ ]:
# Lag-1 (Yesterday's values)
daily_avg['MCP_lag_1'] = daily_avg['MCP (Rs/MWh) '].shift(24)
daily_avg['MCV_lag_1'] = daily_avg['MCV Total (MW)'].shift(24)

daily_avg['MCP_avg'] = daily_avg['MCP (Rs/MWh) '].rolling(window=24).mean()  # Daily average for MCP
daily_avg['MCV_avg'] = daily_avg['MCV Total (MW)'].rolling(window=24).mean()  # Daily average for MCV

daily_avg['MCP_7d_avg'] = daily_avg['MCP (Rs/MWh) '].rolling(window=24 * 7).mean()  # Weekly average for MCP
daily_avg['MCV_7d_avg'] = daily_avg['MCV Total (MW)'].rolling(window=24 * 7).mean()  # Weekly average for MCV

# daily_avg['monthly_avg_MCP'] = daily_avg['MCP (Rs/MWh) '].rolling(window=24 * 30).mean()  # Monthly average for MCP
# daily_avg['monthly_avg_MCV'] = daily_avg['MCV Total (MW)'].rolling(window=24 * 30).mean()  # Monthly average for MCV

# Drop NaN values after adding new columns
daily_avg.dropna(inplace=True)



In [ ]:
daily_avg.head()

In [ ]:
cols = daily_avg.columns.tolist()
# Remove 'Day_of_Week' and 'Month' from their current position and insert them at the beginning
cols.remove('Day_of_Week')
cols.remove('Month')
cols = ['Day_of_Week', 'Month'] + cols
daily_avg = daily_avg[cols]

daily_avg.head()

##correlation


In [ ]:
correlation_matrix = daily_avg.corr()
correlation_matrix

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Matrix of Daily Average Features')
plt.show()

#differant dataframes with strong correlation

In [ ]:
daily_avg['Year'] = daily_avg.index.year

In [ ]:
main_df1 = daily_avg.drop(['Sell bid Solar (MW)', 'Sell bid Non-Solar (MW)', 'Sell bid Hydro (MW)', 'MCV Solar (MW)', 'MCV Non-Solar (MW)', 'MCV Hydro (MW)'], axis=1)

In [ ]:
main_df1.head()

In [ ]:
MCP_df = daily_avg.drop(['Year','Purchase Bid (MW)','Total Sell Bid (MW)', 'Sell bid Solar (MW)', 'Sell bid Non-Solar (MW)', 'Sell bid Hydro (MW)', 'MCV Total (MW)', 'MCV Solar (MW)', 'MCV Non-Solar (MW)', 'MCV Hydro (MW)', 'MCV_lag_1', 'MCV_avg','MCV_7d_avg'], axis = 1)

In [ ]:
MCP_df = MCP_df.drop(['Hour','Month','Day_of_Week'],axis=1)

In [ ]:
MCP_df.head()

In [ ]:
MCV_df = daily_avg.drop(['MCP (Rs/MWh) ','MCV Solar (MW)','MCV Non-Solar (MW)','MCV Hydro (MW)','Hour','Sell bid Hydro (MW)','Sell bid Non-Solar (MW)','Sell bid Solar (MW)'], axis=1)

In [ ]:
MCV_df = daily_avg.drop(['MCP (Rs/MWh) ', 'MCP_lag_1', 'MCP_avg','MCP_7d_avg'], axis=1)

In [ ]:
MCV_df=MCV_df.drop(['Hour','Month','Day_of_Week'],axis=1)

In [ ]:
MCV_df.head()

In [ ]:
# prompt: graph of correlation matrix of MCP_df

import matplotlib.pyplot as plt
import seaborn as sns

# Assuming MCP_df is already defined as in your provided code
correlation_matrix_mcp = MCP_df.corr()

plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix_mcp, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Matrix of MCP_df')
plt.show()

In [ ]:
daily_avg.shape

##preparation for model MCP with MCP correlated

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split,TimeSeriesSplit
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.callbacks import EarlyStopping

In [ ]:
last_month = MCP_df.iloc[-24*31:]

In [ ]:
MCP_df = MCP_df.iloc[:-24*31]

In [ ]:
MCP_df.head()

In [ ]:
MCP_df= MCP_df.drop(['MCP_avg','MCP_7d_avg'],axis=1)

In [ ]:
MCP_df.head()

In [ ]:
X_df = MCP_df
y_df = MCP_df[['MCP (Rs/MWh) ']]

In [ ]:
from sklearn.preprocessing import MinMaxScaler
import joblib
scaler_X = MinMaxScaler()
X_scaled = scaler_X.fit_transform(X_df)
joblib.dump(scaler_X, f'{today_str}_month_mcp_scaler_X.pkl')

In [ ]:
scaler_y = MinMaxScaler()
y_scaled = scaler_y.fit_transform(y_df)
joblib.dump(scaler_y, f'{today_str}_month_mcp_scaler_y.pkl')

In [ ]:
X_data = np.array(X_scaled)
y_data = np.array(y_scaled)

In [ ]:
import numpy as np

def create_sequences(X_data, y_data, sequence_length, prediction_length):
    sequences = []
    labels = []
    for i in range(len(X_data) - sequence_length - prediction_length + 1):
        sequences.append(X_data[i:i + sequence_length])
        labels.append(y_data[i + sequence_length:i + sequence_length + prediction_length])
    return np.array(sequences), np.array(labels)

sequence_length = 24*30
prediction_length = 24*30

# Assuming X_data and y_data are already defined and contain the hourly data
X, y = create_sequences(X_data, y_data, sequence_length, prediction_length)

# Check the shape of the output
print(X.shape)
print(y.shape)


In [ ]:
split_index = int(0.9 * len(X))
X_train, X_test = X[:split_index], X[split_index:]
y_train, y_test = y[:split_index], y[split_index:]
print("Training Data Shape:", X_train.shape, y_train.shape)
print("Testing Data Shape:", X_test.shape, y_test.shape)

## Model for MCP correlated

In [ ]:
sequence_length = X_train.shape[1]
num_features = X_train.shape[2]
output_steps = 24*30
output_units = output_steps * 1

##simple model

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Bidirectional
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

# Define the LSTM model architecture
model_lstm = Sequential()
#model_lstm.add(LSTM(256, return_sequences=True, input_shape=(sequence_length, num_features)))
#model_lstm.add(Dropout(0.1))
model_lstm.add(LSTM(128, return_sequences=True, input_shape=(sequence_length, num_features)))
model_lstm.add(Dropout(0.1))
model_lstm.add(LSTM(64, return_sequences=True))
model_lstm.add(Dropout(0.1))
model_lstm.add(LSTM(32, return_sequences=False))
model_lstm.add(Dropout(0.1))
model_lstm.add(Dense(output_units, activation='linear'))  # MCP and MCV combined output

# Compile the Model
model_lstm.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# Early Stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Model Summary
#model_lstm.summary()

# Train the Model
history_lstm = model_lstm.fit(X_train, y_train.reshape(-1, output_units), epochs=50, batch_size=16,
                              validation_split=0.3, callbacks=[early_stopping])

# Evaluate the Model
train_loss_lstm = model_lstm.evaluate(X_train, y_train.reshape(-1, output_units))
test_loss_lstm = model_lstm.evaluate(X_test, y_test.reshape(-1, output_units))
print(f'Train Loss (LSTM): {train_loss_lstm}')
print(f'Test Loss (LSTM): {test_loss_lstm}')

# Plot the Training and Validation Loss for LSTM
plt.plot(history_lstm.history['loss'], label='Train Loss (LSTM)')
plt.plot(history_lstm.history['val_loss'], label='Validation Loss (LSTM)')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

model_lstm.save('mcv_mcp_lstm_model.h5')

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Bidirectional
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.preprocessing import StandardScaler
import numpy as np
import matplotlib.pyplot as plt

# Scale Input Features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
X_test_scaled = scaler.transform(X_test.reshape(-1, X_test.shape[-1])).reshape(X_test.shape)

# Define the Optimized LSTM Model Architecture
model_lstm = Sequential()
model_lstm.add(Bidirectional(LSTM(128, return_sequences=True, input_shape=(sequence_length, num_features))))
model_lstm.add(Dropout(0.2))
model_lstm.add(Bidirectional(LSTM(64, return_sequences=True)))
model_lstm.add(Dropout(0.2))
model_lstm.add(Bidirectional(LSTM(32, return_sequences=False)))
model_lstm.add(Dropout(0.2))
model_lstm.add(Dense(64, activation='relu'))  # Additional Dense Layer for Better Representation
model_lstm.add(Dense(output_units, activation='linear'))  # MCP and MCV combined output

# Compile the Model with Modified Learning Rate and Optimizer
model_lstm.compile(optimizer=Adam(learning_rate=0.0005), loss='mean_squared_error')

# Add Callbacks: EarlyStopping and ReduceLROnPlateau
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)

# Train the Model
history_lstm = model_lstm.fit(
    X_train_scaled, y_train.reshape(-1, output_units),
    epochs=100,
    batch_size=32,  # Adjusted Batch Size
    validation_split=0.3,
    callbacks=[early_stopping, reduce_lr]
)

# Evaluate the Model
train_loss_lstm = model_lstm.evaluate(X_train_scaled, y_train.reshape(-1, output_units))
test_loss_lstm = model_lstm.evaluate(X_test_scaled, y_test.reshape(-1, output_units))

print(f'Train Loss (LSTM): {train_loss_lstm}')
print(f'Test Loss (LSTM): {test_loss_lstm}')

# Plot the Training and Validation Loss for LSTM
plt.plot(history_lstm.history['loss'], label='Train Loss (LSTM)')
plt.plot(history_lstm.history['val_loss'], label='Validation Loss (LSTM)')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

model_lstm.save('optimized_mcv_mcp_lstm_model.h5')


In [ ]:
from sklearn.metrics import r2_score, mean_squared_error
y_pred_lstm = model_lstm.predict(X_test)

# Inverse transform the scaled predictions and actual values to get the original scale
y_test_original = scaler_y.inverse_transform(y_test.reshape(-1, 1))  # Use scaler_y for y_test
y_pred_lstm_original = scaler_y.inverse_transform(y_pred_lstm.reshape(-1, 1))  # Use scaler_y for predictions

# Calculate the R-squared score
r2_lstm = r2_score(y_test_original, y_pred_lstm_original)

# Calculate Mean Squared Error (MSE)
mse_lstm = mean_squared_error(y_test_original, y_pred_lstm_original)

# Calculate Root Mean Squared Error (RMSE)
rmse_lstm = np.sqrt(mse_lstm)

# Output the evaluation metrics
print(f"LSTM - R-squared: {r2_lstm}, MSE: {mse_lstm}, RMSE: {rmse_lstm}")


In [ ]:
import matplotlib.pyplot as plt

# Assuming y_test_original and y_pred_lstm_original are already defined from your previous code

# Select the last 672 entries (representing the last week)
last_week_actual = y_test_original[-24*30:]
last_week_predicted = y_pred_lstm_original[-24*30:]

# Plot the graph for the last week
plt.figure(figsize=(15, 6))
plt.plot(last_week_actual, label='Actual Values')
plt.plot(last_week_predicted, label='Predicted Values')
plt.xlabel('Time Steps')
plt.ylabel('Scaled Values')
plt.title('Actual vs. Predicted Values for the Last Week (X_test)')
plt.legend()
plt.grid(True)
plt.show()


#MCV Month Ahead


##preparation for model MCV with MCV correlated

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split,TimeSeriesSplit
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.callbacks import EarlyStopping

In [ ]:
X_df = MCV_df
y_df = MCV_df[['MCV Total (MW)']]

In [ ]:
from sklearn.preprocessing import MinMaxScaler
import joblib
scaler_X = MinMaxScaler()
X_scaled = scaler_X.fit_transform(X_df)
joblib.dump(scaler_X, f'{today_str}_mcv_month_scaler_X.pkl')

In [ ]:
scaler_y = MinMaxScaler()
y_scaled = scaler_y.fit_transform(y_df)
joblib.dump(scaler_y, f'{today_str}_mcv_month_scaler_y.pkl')

In [ ]:
X_data = np.array(X_scaled)
y_data = np.array(y_scaled)

In [ ]:
import numpy as np

def create_sequences(X_data, y_data, sequence_length, prediction_length):
    sequences = []
    labels = []
    for i in range(len(X_data) - sequence_length - prediction_length + 1):
        sequences.append(X_data[i:i + sequence_length])
        labels.append(y_data[i + sequence_length:i + sequence_length + prediction_length])
    return np.array(sequences), np.array(labels)

sequence_length = 24*30
prediction_length = 24*30

# Assuming X_data and y_data are already defined and contain the hourly data
X, y = create_sequences(X_data, y_data, sequence_length, prediction_length)

# Check the shape of the output
print(X.shape)
print(y.shape)


In [ ]:
split_index = int(0.9 * len(X))
X_train, X_test = X[:split_index], X[split_index:]
y_train, y_test = y[:split_index], y[split_index:]
print("Training Data Shape:", X_train.shape, y_train.shape)
print("Testing Data Shape:", X_test.shape, y_test.shape)

## Model for MCV correlated

In [ ]:
MCV_df.head()

In [ ]:
MCV_df = MCV_df[['MCV Total (MW)','MCV_lag_1','MCV_avg','MCV_7d_avg']]

In [ ]:
# prompt: graph of correlation matrix of MCV_df

import matplotlib.pyplot as plt
import seaborn as sns

# Assuming MCV_df is already defined as in your provided code
correlation_matrix_mcv = MCV_df.corr()

plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix_mcv, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Matrix of MCV_df')
plt.show()

In [ ]:
last_month = MCV_df.iloc[-24*31:]
MCV_df = MCV_df.iloc[:-24*31]
X_df = MCV_df
y_df = MCV_df[['MCV Total (MW)']]

In [ ]:
from sklearn.preprocessing import MinMaxScaler
import joblib
scaler_X = MinMaxScaler()
X_scaled = scaler_X.fit_transform(X_df)
joblib.dump(scaler_X, f'{today_str}_mcv_month_scaler_X.pkl')
scaler_y = MinMaxScaler()
y_scaled = scaler_y.fit_transform(y_df)
joblib.dump(scaler_y, f'{today_str}_mcv_month_scaler_y.pkl')
X_data = np.array(X_scaled)
y_data = np.array(y_scaled)

In [ ]:
import numpy as np

def create_sequences(X_data, y_data, sequence_length, prediction_length):
    sequences = []
    labels = []
    for i in range(len(X_data) - sequence_length - prediction_length + 1):
        sequences.append(X_data[i:i + sequence_length])
        labels.append(y_data[i + sequence_length:i + sequence_length + prediction_length])
    return np.array(sequences), np.array(labels)

sequence_length = 24*30
prediction_length = 24*30

# Assuming X_data and y_data are already defined and contain the hourly data
X, y = create_sequences(X_data, y_data, sequence_length, prediction_length)

# Check the shape of the output
print(X.shape)
print(y.shape)


In [ ]:
split_index = int(0.9 * len(X))
X_train, X_test = X[:split_index], X[split_index:]
y_train, y_test = y[:split_index], y[split_index:]
print("Training Data Shape:", X_train.shape, y_train.shape)
print("Testing Data Shape:", X_test.shape, y_test.shape)

In [ ]:
sequence_length = X_train.shape[1]
num_features = X_train.shape[2]
output_steps = 24*30
output_units = output_steps * 1

##tune

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Bidirectional
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
from keras.regularizers import l2

model_lstm = Sequential()
model_lstm.add(LSTM(128, return_sequences=True, input_shape=(sequence_length, num_features),
                    kernel_regularizer=l2(0.001)))  # L2 regularization
model_lstm.add(Dropout(0.2))  # Increased dropout
model_lstm.add(LSTM(64, return_sequences=True, kernel_regularizer=l2(0.001)))  # Regularization
model_lstm.add(Dropout(0.2))  # Increased dropout
model_lstm.add(LSTM(32, return_sequences=False, kernel_regularizer=l2(0.001)))  # Regularization
model_lstm.add(Dropout(0.2))  # Increased dropout
model_lstm.add(Dense(output_units, activation='linear'))

# Compile the Model with lower learning rate
model_lstm.compile(optimizer=Adam(learning_rate=0.0005), loss='mean_squared_error')

# Early Stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Train the Model
history_lstm = model_lstm.fit(X_train, y_train.reshape(-1, output_units), epochs=50, batch_size=16,
                              validation_split=0.3, callbacks=[early_stopping])


# Evaluate the Model
train_loss_lstm = model_lstm.evaluate(X_train, y_train.reshape(-1, output_units))
test_loss_lstm = model_lstm.evaluate(X_test, y_test.reshape(-1, output_units))
print(f'Train Loss (LSTM): {train_loss_lstm}')
print(f'Test Loss (LSTM): {test_loss_lstm}')

# Plot the Training and Validation Loss for LSTM
plt.plot(history_lstm.history['loss'], label='Train Loss (LSTM)')
plt.plot(history_lstm.history['val_loss'], label='Validation Loss (LSTM)')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

model_lstm.save('mcV_month_lstm_model.h5')

##HYPERTUNE

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import mixed_precision
import matplotlib.pyplot as plt
import gc
from keras import backend as K

In [ ]:
# Enable mixed precision for better memory usage
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)

# Data generator
def data_generator(X, y, batch_size):
    """Generator to yield batches of data for model training."""
    num_samples = len(X)
    while True:
        indices = np.random.permutation(num_samples)  # Shuffle indices
        for i in range(0, num_samples, batch_size):
            batch_indices = indices[i:i + batch_size]
            X_batch = X[batch_indices]
            y_batch = y[batch_indices]
            yield X_batch, y_batch
            del X_batch, y_batch  # Clear batch to free memory

# Clear session and run garbage collection after training
def clear_session_and_gc():
    K.clear_session()
    gc.collect()

# Batch size
batch_size = 16  # Reduced batch size for memory efficiency
train_gen = data_generator(X_train, y_train.reshape(-1, output_units), batch_size)
steps_per_epoch = len(X_train) // batch_size

# Define the LSTM Model
model_lstm = Sequential()
model_lstm.add(LSTM(64, return_sequences=True, input_shape=(sequence_length, num_features)))
model_lstm.add(Dropout(0.2))
model_lstm.add(LSTM(32, return_sequences=True))
model_lstm.add(Dropout(0.2))
model_lstm.add(LSTM(16, return_sequences=False))
model_lstm.add(Dropout(0.2))
model_lstm.add(Dense(output_units, activation='linear'))

# Compile the model
model_lstm.compile(optimizer=Adam(learning_rate=0.005), loss='mean_squared_error')

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Model summary
model_lstm.summary()

# Train the Model
history_lstm = model_lstm.fit(
    train_gen,
    steps_per_epoch=steps_per_epoch,
    epochs=50,
    validation_data=(X_test, y_test.reshape(-1, output_units)),
    callbacks=[early_stopping]
)

# Evaluate the Model
train_loss_lstm = model_lstm.evaluate(X_train, y_train.reshape(-1, output_units))
test_loss_lstm = model_lstm.evaluate(X_test, y_test.reshape(-1, output_units))

print(f'Train Loss (LSTM): {train_loss_lstm}')
print(f'Test Loss (LSTM): {test_loss_lstm}')

# Plot Training and Validation Loss
plt.plot(history_lstm.history['loss'], label='Train Loss (LSTM)')
plt.plot(history_lstm.history['val_loss'], label='Validation Loss (LSTM)')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Save the model
model_lstm.save('mcv_mcp_lstm_model.h5')

# Clear session and garbage collection after training
clear_session_and_gc()


In [ ]:
from sklearn.metrics import r2_score, mean_squared_error
y_pred_lstm = model_lstm.predict(X_test)

# Inverse transform the scaled predictions and actual values to get the original scale
y_test_original = scaler_y.inverse_transform(y_test.reshape(-1, 1))  # Use scaler_y for y_test
y_pred_lstm_original = scaler_y.inverse_transform(y_pred_lstm.reshape(-1, 1))  # Use scaler_y for predictions

# Calculate the R-squared score
r2_lstm = r2_score(y_test_original, y_pred_lstm_original)

# Calculate Mean Squared Error (MSE)
mse_lstm = mean_squared_error(y_test_original, y_pred_lstm_original)

# Calculate Root Mean Squared Error (RMSE)
rmse_lstm = np.sqrt(mse_lstm)

# Output the evaluation metrics
print(f"LSTM - R-squared: {r2_lstm}, MSE: {mse_lstm}, RMSE: {rmse_lstm}")


In [ ]:
import matplotlib.pyplot as plt

# Assuming y_test_original and y_pred_lstm_original are already defined from your previous code

# Select the last 672 entries (representing the last week)
last_week_actual = y_test_original[-24*30:]
last_week_predicted = y_pred_lstm_original[-24*30:]

# Plot the graph for the last week
plt.figure(figsize=(15, 6))
plt.plot(last_week_actual, label='Actual Values')
plt.plot(last_week_predicted, label='Predicted Values')
plt.xlabel('Time Steps')
plt.ylabel('Scaled Values')
plt.title('Actual vs. Predicted Values for (X_test)')
plt.legend()
plt.grid(True)
plt.show()


##LSTM

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Bidirectional
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

# Define the LSTM model architecture
model_lstm = Sequential()
#model_lstm.add(LSTM(256, return_sequences=True, input_shape=(sequence_length, num_features)))
#model_lstm.add(Dropout(0.1))
model_lstm.add(LSTM(128, return_sequences=True, input_shape=(sequence_length, num_features)))
model_lstm.add(Dropout(0.1))
model_lstm.add(LSTM(64, return_sequences=True))
model_lstm.add(Dropout(0.1))
model_lstm.add(LSTM(32, return_sequences=False))
model_lstm.add(Dropout(0.1))
model_lstm.add(Dense(output_units, activation='linear'))  # MCP and MCV combined output

# Compile the Model
model_lstm.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# Early Stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Model Summary
#model_lstm.summary()

# Train the Model
history_lstm = model_lstm.fit(X_train, y_train.reshape(-1, output_units), epochs=50, batch_size=16,
                              validation_split=0.3, callbacks=[early_stopping])

# Evaluate the Model
train_loss_lstm = model_lstm.evaluate(X_train, y_train.reshape(-1, output_units))
test_loss_lstm = model_lstm.evaluate(X_test, y_test.reshape(-1, output_units))
print(f'Train Loss (LSTM): {train_loss_lstm}')
print(f'Test Loss (LSTM): {test_loss_lstm}')

# Plot the Training and Validation Loss for LSTM
plt.plot(history_lstm.history['loss'], label='Train Loss (LSTM)')
plt.plot(history_lstm.history['val_loss'], label='Validation Loss (LSTM)')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

model_lstm.save('mcv_mcp_lstm_model.h5')